In [3]:
import pandas as pd
from pyathena import connect
from dotenv import load_dotenv
from script.parse_sql import parse_sql_file
load_dotenv()

True

In [5]:
def cursor_to_dataframe(cursor):

    column_list = [row[0] for row in cursor.description]

    return pd.DataFrame(data=cursor.fetchall(), columns=column_list)


cursor = connect(s3_staging_dir="s3://mentha-athena-query-result/sample",
                 region_name="ap-northeast-2",
                 schema_name="data_store"
).cursor()

# 쿼리 출력 및 csv 저장

In [6]:
for i in range(1, 10):

    query = parse_sql_file(f"sql/query-{i}.sql")
    cursor.execute(query)
    query_result = cursor_to_dataframe(cursor)

    print(query.split("\n")[0])
    print(query_result.head(5), end="\n\n")

    query_result.to_csv(f"csv/query-{i}.csv", index=False, encoding="utf-8")

/* 1. 일별 Active User 데이터 추출 */
         date  acitve_user
0  2018-05-01         2830
1  2018-05-02       333893
2  2018-05-03       341539
3  2018-05-04       350451
4  2018-05-05       350957

/* 2. 일별 구매 유저 및 구매 금액 추출 */
         date  acitve_user      amount
0  2018-05-01         2830    236000.0
1  2018-05-02       333893  31058000.0
2  2018-05-03       341539  31484000.0
3  2018-05-04       350451  32048000.0
4  2018-05-05       350957  32176000.0

/* 3. 이벤트별 User 데이터 추출 */
          event_name  acitve_user
0      abx:firstopen      2858879
1  abx:start_session      1981847
2          abx:login      1514545
3      custom:battle      1257559
4    abx:end_session      1078501

/* 4. 캠페인별 User 데이터 추출 */
    campaign  acitve_user
0  campaign1       250711
1  campaign2       312656
2  campaign3        65078
3       None      2386560

/* 5. 캠페인별 구매 금액 추출 */
    campaign  acitve_user amount
0  campaign1       250711    0.0
1  campaign2       312656    0.0
2  campaign3        65078    0.0

## Test: 일별 Active User 데이터 추출

In [7]:
query1 = parse_sql_file("sql/query-1.sql")
cursor.execute(query1)
query1_result = cursor_to_dataframe(cursor)
query1_result.head(5)

,date,acitve_user
0,2018-05-01,2830
1,2018-05-02,333893
2,2018-05-03,341539
3,2018-05-04,350451
4,2018-05-05,350957
